Input File Creation
===========
First let's start with some tools to create input files for a given deployment schedule.

In [18]:
import os
import uuid
import json
import subprocess
from math import ceil
from copy import deepcopy

import numpy as np
import pandas as pd
import cymetric as cym
%matplotlib inline
import matplotlib.pyplot as plt
import george

import dtw

In [2]:
with open('once-through.json') as f:
    BASE_SIM = json.load(f)
DURATION = BASE_SIM['simulation']['control']['duration']
YEARS = ceil(DURATION / 12)
MONTH_SHUFFLE = (1, 7, 10, 4, 8, 6, 12, 2, 5, 9, 11, 3)
NULL_SCHEDULE = {'build_times': [{'val': 1}], 
                 'n_build': [{'val': 0}], 
                 'prototypes': [{'val': 'LWR'}]}
LWR_PROTOTYPE = {'val': 'LWR'}
OPT_H5 = 'opt.h5'

In [3]:
BASE_SIM['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

In [4]:
def deploy_inst_schedule(Θ):
    if np.sum(Θ) == 0: 
        return NULL_SCHEDULE
    sched = {'build_times': {'val': []},
             'n_build': {'val': []},
             'prototypes': {'val': []}}
    build_times = sched['build_times']['val']
    n_build = sched['n_build']['val']
    prototypes = sched['prototypes']['val']
    m = 0
    for i, θ in enumerate(Θ):
        if θ <= 0:
            continue
        build_times.append(i*12 + MONTH_SHUFFLE[m])
        n_build.append(int(θ))
        prototypes.append('LWR')
        m = (m + 1) % 12
    return sched

def make_sim(Θ, fname='sim.json'):
    sim = deepcopy(BASE_SIM)
    inst = sim['simulation']['region']['institution']
    inst['config']['DeployInst'] = deploy_inst_schedule(Θ)
    with open(fname, 'w') as f:
        json.dump(sim, f)
    return sim

In [5]:
s = make_sim([])

In [6]:
s['simulation']['region']['institution']['config']['DeployInst']

{'build_times': [{'val': 1}],
 'n_build': [{'val': 0}],
 'prototypes': [{'val': 'LWR'}]}

Simulate
=========
Now let's build some tools to run simulations and extract a GWe time series.

In [7]:
def run(fname='sim.json', out=OPT_H5):
    """Runs a simulation and returns the sim id."""
    cmd = ['cyclus', '--warn-limit', '0', '-o', out, fname]
    proc = subprocess.run(cmd, check=True, universal_newlines=True, stdout=subprocess.PIPE)
    simid = proc.stdout.rsplit(None, 1)[-1]
    return simid

ZERO_GWE = pd.DataFrame({'GWe': np.zeros(YEARS)}, index=np.arange(YEARS))
ZERO_GWE.index.name = 'Time'

def extract_gwe(simid, out=OPT_H5):
    """Computes the annual GWe for a simulation."""
    with cym.dbopen(out) as db:
        evaler = cym.Evaluator(db)
        raw = evaler.eval('TimeSeriesPower', conds=[('SimId', '==', uuid.UUID(simid))])
    ano = pd.DataFrame({'Time': raw.Time.apply(lambda x: x//12), 
                        'GWe': raw.Value.apply(lambda x: 1e-3*x/12)})
    gwe = ano.groupby('Time').sum()
    gwe = (gwe + ZERO_GWE).fillna(0.0)
    return np.array(gwe.GWe)

Distancing
========
Now let's build some tools to distance between a GWe time series and a demand curve.

In [8]:
DEFAULT_DEMAND = 90 * (1.01**np.arange(YEARS))  # 1% growth

In [9]:
def d(g, f=None):
    """The dynamic time warping distance between a GWe time series and a demand curve."""
    f = DEFAULT_DEMAND if f is None else f
    rtn = dtw.distance(f[:, np.newaxis], g[:, np.newaxis])
    return rtn

def gwed(Θ, f=None):
    """For a given deployment schedule Θ, return the GWe time series and the distance 
    to the demand function f.
    """
    make_sim(Θ)
    simid = run()
    gwe = extract_gwe(simid)
    dΘ = d(gwe, f=f)
    return gwe, dΘ

Initialize Optimization
===============
Now let's start with a couple of simple simulations

In [10]:
N = np.asarray(np.ceil(4*(1.01)**np.arange(YEARS)), dtype=int)  # max annual deployments
Θs = [] # deployment schedules
G = []  # GWe per sim
D = []  # distances per sim
if os.path.isfile(OPT_H5):
    os.remove(OPT_H5)

In [11]:
def add_sim(Θ, f=None):
    """Add a simulation to the known simulations by performing the simulation."""
    g_s, d_s = gwed(Θ, f=f)
    Θs.append(Θ)
    G.append(g_s)
    D.append(d_s)

First, add a schedule where nothing is deployed, leaving the initial facilities to retire.

In [12]:
add_sim(np.zeros(YEARS, dtype=int))

Next, add a simulation that is the max deployment schedule to bound the space

In [13]:
add_sim(N)

Optimizer
=======
Now let's add some tools to do the estimation phase of the optimization.

In [78]:
Γ = 100

In [82]:
def gp_gwe(Θs, G, T=None, tol=1e-6, verbose=False):
    """Create a Gaussian process regression model for GWe."""
    S = len(G)
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θs[0])
    ndim = P + 1
    y_mean = np.mean(G)
    y = np.concatenate(G)
    x = np.empty((S*T, ndim), dtype=int)
    for i in range(S):
        x[i*T:(i+1)*T, 0] = t
        x[i*T:(i+1)*T, 1:] = Θs[i][np.newaxis, :]
    yerr = tol * y_mean
    kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def predict_gwe(Θ, gp, y, T=None):
    """Predict GWe for a deployment schedule Θ and a GP."""
    T = YEARS if T is None else T
    t = np.arange(T)
    P = len(Θ)
    ndim = P + 1
    x = np.empty((T, ndim), dtype=int)
    x[:,0] = t
    x[:,1:] = Θ[np.newaxis,:]
    mu = gp.predict(y, x, mean_only=True)
    return mu

def gp_d_inv(θ_p, D_inv, tol=1e-6, verbose=False):
    """Computes a Gaussian process model for a deployment parameter."""
    S = len(D)
    ndim = 1
    x = θ_p
    y = D_inv
    y_mean = np.mean(y)
    yerr = tol * y_mean
    kernel = float(y_mean) * george.kernels.ExpSquaredKernel(1.0, ndim=ndim)
    gp = george.GP(kernel, mean=y_mean)
    gp.compute(x, yerr=yerr, sort=False)
    gp.optimize(x, y, yerr=yerr, sort=False, verbose=verbose)
    return gp, x, y

def weights(Θs, D, N, tol=1e-6, verbose=False):
    P = len(N)
    θ_ps = np.array(Θs)
    D_inv = np.array(D)**-1
    W = []
    for p in range(P):
        θ_p = θ_ps[:,p]
        gp, _, _ = gp_d_inv(θ_p, D_inv, tol=tol, verbose=verbose)
        d_np = gp.predict(D_inv, np.arange(0, N[p] + 1), mean_only=True)
        d_np_tot = d_np.sum()
        w_p = d_np / d_np_tot
        W.append(w_p)
    return W

def guess_scheds(W, Γ, gp, y, T=None):
    """Guess a new deployment schedule, given a number of samples Γ, weights W, and 
    Guassian process for the GWe.
    """
    P = len(W)
    Θ_γs = np.empty((Γ, P), dtype=int)
    for p in range(P):
        w_p = W[p]
        Θ_γs[:, p] = np.random.choice(len(w_p), size=Γ, p=w_p)
    Δ = []
    for γ in range(Γ):
        Θ_γ = Θ_γs[γ]
        g_star = predict_gwe(Θ_γ, gp, y, T=T)
        d_star = d(g_star)
        Δ.append(d_star)
    γ = np.argmin(Δ)
    Θ_γ = Θ_γs[γ]
    return Θ_γ

In [58]:
gp, x, y = gp_gwe(Θs, G)

In [65]:
W = weights(Θs, D, N)

In [84]:
guess_scheds(W, Γ, gp, y)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True], dtype=bool)

In [74]:
np.random.choice(len(W[0]), size=100, p=W[0])

array([2, 2, 4, 2, 0, 4, 1, 2, 2, 4, 2, 1, 4, 3, 3, 3, 2, 4, 1, 1, 3, 2, 2,
       3, 1, 0, 4, 3, 1, 3, 2, 2, 3, 0, 3, 3, 2, 2, 0, 4, 3, 0, 0, 3, 4, 4,
       2, 4, 1, 0, 4, 3, 1, 1, 3, 2, 0, 2, 3, 0, 4, 1, 0, 3, 1, 3, 2, 4, 3,
       1, 3, 4, 1, 3, 2, 2, 2, 2, 3, 0, 4, 1, 1, 1, 1, 3, 4, 3, 3, 2, 0, 0,
       0, 4, 3, 3, 3, 4, 1, 1])

In [77]:
len(W)

50

In [81]:
np.empty((4, 2))[0, :]

array([ 0.,  0.])